In [1]:
import geopandas as gpd
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib.colors as mcolors

/Users/user/miniconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
file_path = "analysis_oct06/noderisk/"
plot_title = 'County infection risk by end of autumn '+r'(seed=730605)'
fig_path = file_path+"/county_risk_seed730605_end_of_autumn.png"
risk_file = file_path+"/noderisk_end_of_autumn_730605.csv"
print(plot_title)

County infection risk by end of autumn (seed=730605)


In [3]:
# Read node_risk file
node_risk = pd.read_csv(risk_file)
node_risk

,Node,Risk
0,726636,0.061628
1,726687,0.061578
2,726791,0.061450
3,726410,0.061400
4,726478,0.061350
...,...,...
63979,730384,0.000000
63980,730373,0.000000
63981,730352,0.000000
63982,731703,0.000000


In [4]:
# Load location data
location_df = pd.read_csv('data/cattle_FLAPS_for_USAMMv3.txt', sep = "\t")
location_df = location_df.loc[location_df.type == "d"]
location_df = location_df[['Id', 'Lat', 'Lon', 'd']]
location_df

,Id,Lat,Lon,d
53,757624,32.590559,-86.803342,16
118,757844,32.591639,-86.467205,13
126,757842,32.543002,-86.472609,15
229,756096,30.615888,-87.854986,4
245,756766,30.393238,-87.517768,2
...,...,...,...,...
815615,729298,43.910056,-108.098869,2
815616,729318,43.933834,-108.010241,2
815749,730103,43.857095,-104.114942,4
815781,730094,44.100281,-104.130074,7


In [5]:
# Merge the dataframes on the location id
merged_df = pd.merge(location_df, node_risk, left_on = 'Id', right_on = 'Node', how = 'left')
merged_df

,Id,Lat,Lon,d,Node,Risk
0,757624,32.590559,-86.803342,16,757624,0.0
1,757844,32.591639,-86.467205,13,757844,0.0
2,757842,32.543002,-86.472609,15,757842,0.0
3,756096,30.615888,-87.854986,4,756096,0.0
4,756766,30.393238,-87.517768,2,756766,0.0
...,...,...,...,...,...,...
63979,729298,43.910056,-108.098869,2,729298,0.0
63980,729318,43.933834,-108.010241,2,729318,0.0
63981,730103,43.857095,-104.114942,4,730103,0.0
63982,730094,44.100281,-104.130074,7,730094,0.0


In [6]:
merged_df = merged_df.drop(columns=['Id'])
merged_df

,Lat,Lon,d,Node,Risk
0,32.590559,-86.803342,16,757624,0.0
1,32.591639,-86.467205,13,757844,0.0
2,32.543002,-86.472609,15,757842,0.0
3,30.615888,-87.854986,4,756096,0.0
4,30.393238,-87.517768,2,756766,0.0
...,...,...,...,...,...
63979,43.910056,-108.098869,2,729298,0.0
63980,43.933834,-108.010241,2,729318,0.0
63981,43.857095,-104.114942,4,730103,0.0
63982,44.100281,-104.130074,7,730094,0.0


In [7]:
# Create a GeoDataFrame from the merged DataFrame
plot_gdf = gpd.GeoDataFrame(merged_df, geometry=gpd.points_from_xy(merged_df['Lon'], merged_df['Lat']))
plot_gdf

,Lat,Lon,d,Node,Risk,geometry
0,32.590559,-86.803342,16,757624,0.0,POINT (-86.80334 32.59056)
1,32.591639,-86.467205,13,757844,0.0,POINT (-86.46721 32.59164)
2,32.543002,-86.472609,15,757842,0.0,POINT (-86.47261 32.54300)
3,30.615888,-87.854986,4,756096,0.0,POINT (-87.85499 30.61589)
4,30.393238,-87.517768,2,756766,0.0,POINT (-87.51777 30.39324)
...,...,...,...,...,...,...
63979,43.910056,-108.098869,2,729298,0.0,POINT (-108.09887 43.91006)
63980,43.933834,-108.010241,2,729318,0.0,POINT (-108.01024 43.93383)
63981,43.857095,-104.114942,4,730103,0.0,POINT (-104.11494 43.85710)
63982,44.100281,-104.130074,7,730094,0.0,POINT (-104.13007 44.10028)


In [8]:
# Load the shapefiles
shapefile_county = 'data/gadm41_USA_shp/gadm41_USA_2.shp'  
counties_sf = gpd.read_file(shapefile_county)
counties_sf = counties_sf.cx[-130:-60, 25:50]

shapefile_state = 'data/gadm41_USA_shp/gadm41_USA_1.shp'  
states_sf = gpd.read_file(shapefile_state)
states_sf = states_sf.cx[-130:-60, 25:50]

In [9]:
# Step 3: Perform spatial join with counties to assign each premises to a county
premises_with_county = gpd.sjoin(plot_gdf, counties_sf, how="left", op='intersects')
premises_with_county

/Users/user/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/z0/wy551r2j0yx_cykzvy1n6hq40000gp/T/ipykernel_43091/1952189627.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  premises_with_county = gpd.sjoin(plot_gdf, counties_sf, how="left", op='intersects')


,Lat,Lon,d,Node,Risk,geometry,index_right,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2
0,32.590559,-86.803342,16,757624,0.0,POINT (-86.80334 32.59056),0.0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU
1,32.591639,-86.467205,13,757844,0.0,POINT (-86.46721 32.59164),0.0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU
2,32.543002,-86.472609,15,757842,0.0,POINT (-86.47261 32.54300),0.0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU
3,30.615888,-87.854986,4,756096,0.0,POINT (-87.85499 30.61589),1.0,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD
4,30.393238,-87.517768,2,756766,0.0,POINT (-87.51777 30.39324),1.0,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63979,43.910056,-108.098869,2,729298,0.0,POINT (-108.09887 43.91006),3146.0,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA
63980,43.933834,-108.010241,2,729318,0.0,POINT (-108.01024 43.93383),3146.0,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA
63981,43.857095,-104.114942,4,730103,0.0,POINT (-104.11494 43.85710),3147.0,USA.51.23_1,USA,United States,USA.51_1,Wyoming,NA,Weston,NA,NA,County,County,NA,US.WY.WE
63982,44.100281,-104.130074,7,730094,0.0,POINT (-104.13007 44.10028),3147.0,USA.51.23_1,USA,United States,USA.51_1,Wyoming,NA,Weston,NA,NA,County,County,NA,US.WY.WE


In [10]:
# Compute the weighted sum and sum of weights (cattle counts)
premises_with_county['weighted_risk'] = premises_with_county['Risk'] * premises_with_county['d']
county_risk = premises_with_county.groupby(['NAME_1', 'NAME_2']).agg(
    total_weighted_risk=('weighted_risk', 'sum'),
    total_cattle=('d', 'sum')
).reset_index()
premises_with_county.head()

,Lat,Lon,d,Node,Risk,geometry,index_right,GID_2,GID_0,COUNTRY,...,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,weighted_risk
0,32.590559,-86.803342,16,757624,0.0,POINT (-86.80334 32.59056),0.0,USA.1.1_1,USA,United States,...,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,0.0
1,32.591639,-86.467205,13,757844,0.0,POINT (-86.46721 32.59164),0.0,USA.1.1_1,USA,United States,...,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,0.0
2,32.543002,-86.472609,15,757842,0.0,POINT (-86.47261 32.54300),0.0,USA.1.1_1,USA,United States,...,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,0.0
3,30.615888,-87.854986,4,756096,0.0,POINT (-87.85499 30.61589),1.0,USA.1.2_1,USA,United States,...,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD,0.0
4,30.393238,-87.517768,2,756766,0.0,POINT (-87.51777 30.39324),1.0,USA.1.2_1,USA,United States,...,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD,0.0


In [20]:

# Calculate the weighted average
county_risk['weighted_avg_risk'] = county_risk['total_weighted_risk'] / county_risk['total_cattle']
county_risk.sort_values(by = 'weighted_avg_risk', ascending = False).head(100)

,NAME_1,NAME_2,total_weighted_risk,total_cattle,weighted_avg_risk
144,California,Tulare,53453.717622,891885,0.059933
109,California,Kings,15381.451622,307048,0.050095
117,California,Merced,25051.333539,519806,0.048194
2072,Texas,Castro,3270.913189,68898,0.047475
103,California,Fresno,9342.872722,204447,0.045698
...,...,...,...,...,...
106,California,Imperial,8.308222,3052,0.002722
214,Florida,Bradford,1.782133,656,0.002717
2225,Utah,Sanpete,31.723078,11775,0.002694
2338,Washington,Cowlitz,3.300350,1272,0.002595


In [12]:
# Step 5: Merge the weighted average risk data back to the counties GeoDataFrame for mapping or further analysis
counties_with_risk = counties_sf.merge(county_risk[['NAME_1', 'NAME_2', 'weighted_avg_risk']], 
                                    on=['NAME_1', 'NAME_2'], how='left')
counties_with_risk.sort_values(by = 'weighted_avg_risk', ascending = False)

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,weighted_avg_risk
210,USA.5.54_1,USA,United States,USA.5_1,California,NA,Tulare,NA,NA,County,County,NA,US.CA.TL,"POLYGON ((-119.57764 36.48855, -119.47060 36.5...",0.059933
172,USA.5.16_1,USA,United States,USA.5_1,California,NA,Kings,NA,NA,County,County,NA,US.CA.KI,"POLYGON ((-120.31820 35.90518, -120.29678 35.9...",0.050095
180,USA.5.24_1,USA,United States,USA.5_1,California,NA,Merced,NA,NA,County,County,NA,US.CA.MD,"POLYGON ((-120.54173 37.04589, -120.89619 36.7...",0.048194
2530,USA.44.35_1,USA,United States,USA.44_1,Texas,NA,Castro,NA,NA,County,County,NA,US.TX.AS,"POLYGON ((-102.52688 34.74675, -102.17270 34.7...",0.047475
166,USA.5.10_1,USA,United States,USA.5_1,California,NA,Fresno,NA,NA,County,County,NA,US.CA.FR,"POLYGON ((-120.91826 36.74143, -120.91035 36.7...",0.045698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3061,USA.50.42_1,USA,United States,USA.50_1,Wisconsin,NA,Menominee,NA,NA,County,County,NA,US.WI.MN,"POLYGON ((-88.98120 45.02985, -88.98181 45.117...",NaN
3062,USA.50.43_1,USA,United States,USA.50_1,Wisconsin,NA,Milwaukee,NA,NA,County,County,NA,US.WI.MU,"POLYGON ((-87.89491 43.19473, -87.89367 43.193...",NaN
3065,USA.50.46_1,USA,United States,USA.50_1,Wisconsin,NA,Oneida,NA,NA,County,County,NA,US.WI.ON,"POLYGON ((-89.04769 45.89526, -89.04696 45.809...",NaN
3085,USA.50.66_1,USA,United States,USA.50_1,Wisconsin,NA,Vilas,NA,NA,County,County,NA,US.WI.VI,"POLYGON ((-89.92905 46.30005, -89.75987 46.267...",NaN


In [13]:
#Source
plot_gdf_source = plot_gdf[plot_gdf.Node == 730575]
plot_gdf_source

,Lat,Lon,d,Node,Risk,geometry
47402,34.585765,-102.211606,15164,730575,0.055961,POINT (-102.21161 34.58577)


In [14]:
# Separate nodes with risk 0 and risk > 0
counties_with_risk_0 = counties_with_risk[counties_with_risk['weighted_avg_risk'] == 0]
counties_with_risk_0

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,weighted_avg_risk
0,USA.1.1_1,USA,United States,USA.1_1,Alabama,NA,Autauga,NA,NA,County,County,NA,US.AL.AU,"POLYGON ((-86.91668 32.66431, -86.82678 32.660...",0.0
1,USA.1.2_1,USA,United States,USA.1_1,Alabama,NA,Baldwin,NA,NA,County,County,NA,US.AL.BD,"MULTIPOLYGON (((-87.74416 30.29445, -87.74389 ...",0.0
2,USA.1.3_1,USA,United States,USA.1_1,Alabama,NA,Barbour,NA,NA,County,County,NA,US.AL.BR,"POLYGON ((-85.41607 31.61962, -85.49133 31.620...",0.0
4,USA.1.5_1,USA,United States,USA.1_1,Alabama,NA,Blount,NA,NA,County,County,NA,US.AL.BU,"POLYGON ((-86.96287 33.85858, -86.96512 33.863...",0.0
6,USA.1.7_1,USA,United States,USA.1_1,Alabama,NA,Butler,NA,NA,County,County,NA,US.AL.BT,"POLYGON ((-86.70188 31.52521, -86.83881 31.526...",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3110,USA.51.17_1,USA,United States,USA.51_1,Wyoming,NA,Sheridan,NA,NA,County,County,NA,US.WY.SH,"POLYGON ((-107.91039 45.00080, -107.89550 45.0...",0.0
3112,USA.51.19_1,USA,United States,USA.51_1,Wyoming,NA,Sweetwater,NA,NA,County,County,NA,US.WY.SW,"POLYGON ((-107.91540 41.00295, -107.96598 41.0...",0.0
3114,USA.51.21_1,USA,United States,USA.51_1,Wyoming,NA,Uinta,NA,NA,County,County,NA,US.WY.UI,"POLYGON ((-111.04584 41.25143, -111.04594 41.3...",0.0
3115,USA.51.22_1,USA,United States,USA.51_1,Wyoming,NA,Washakie,NA,NA,County,County,NA,US.WY.WA,"POLYGON ((-108.55055 44.16851, -107.14581 44.1...",0.0


In [15]:
counties_with_risk_nonzero = counties_with_risk[counties_with_risk['weighted_avg_risk'] > 0]
counties_with_risk_nonzero.sort_values(by = 'weighted_avg_risk', ascending = False)

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry,weighted_avg_risk
210,USA.5.54_1,USA,United States,USA.5_1,California,NA,Tulare,NA,NA,County,County,NA,US.CA.TL,"POLYGON ((-119.57764 36.48855, -119.47060 36.5...",5.993342e-02
172,USA.5.16_1,USA,United States,USA.5_1,California,NA,Kings,NA,NA,County,County,NA,US.CA.KI,"POLYGON ((-120.31820 35.90518, -120.29678 35.9...",5.009462e-02
180,USA.5.24_1,USA,United States,USA.5_1,California,NA,Merced,NA,NA,County,County,NA,US.CA.MD,"POLYGON ((-120.54173 37.04589, -120.89619 36.7...",4.819362e-02
2530,USA.44.35_1,USA,United States,USA.44_1,Texas,NA,Castro,NA,NA,County,County,NA,US.TX.AS,"POLYGON ((-102.52688 34.74675, -102.17270 34.7...",4.747472e-02
166,USA.5.10_1,USA,United States,USA.5_1,California,NA,Fresno,NA,NA,County,County,NA,US.CA.FR,"POLYGON ((-120.91826 36.74143, -120.91035 36.7...",4.569826e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520,USA.26.66_1,USA,United States,USA.26_1,Missouri,NA,Miller,NA,NA,County,County,NA,US.MO.ML,"POLYGON ((-92.40553 38.02437, -92.51713 38.026...",2.666667e-07
2966,USA.49.2_1,USA,United States,USA.49_1,West Virginia,NA,Berkeley,NA,NA,County,County,NA,US.WV.BE,"POLYGON ((-78.02672 39.62414, -78.02581 39.622...",2.619582e-07
962,USA.18.2_1,USA,United States,USA.18_1,Kentucky,NA,Allen,NA,NA,County,County,NA,US.KY.AL,"POLYGON ((-85.97704 36.72255, -85.97700 36.715...",1.786352e-07
1754,USA.31.10_1,USA,United States,USA.31_1,New Jersey,NA,Hunterdon,NA,NA,County,County,NA,US.NJ.HN,"POLYGON ((-74.74840 40.42410, -74.80936 40.416...",1.185818e-07


In [16]:
# Step 2: Normalize the 'weighted_avg_risk' column
# Normalize using Min-Max normalization to bring the values between 0 and 1
min_risk = counties_with_risk['weighted_avg_risk'].min()
max_risk = counties_with_risk['weighted_avg_risk'].max()

epsilon = 0.1
# Avoid division by zero if all values are the same
if max_risk != min_risk:
    counties_with_risk['normalized_risk'] = epsilon + (1-epsilon) * (
        (counties_with_risk['weighted_avg_risk'] - min_risk) /
        (max_risk - min_risk)
    )
else:
    counties_with_risk['normalized_risk'] = 1  # If all values are the same, set them to 1


In [3]:
# Step 3: Plot the map using the normalized risk values
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

counties_sf.plot(ax=ax, color='white', edgecolor='grey', alpha = 0.3)
states_sf.plot(ax=ax, color='white', edgecolor='black', alpha = 0.5)
# Plot counties using 'normalized_risk' for the color
counties_with_risk.plot(
    column='normalized_risk',    # Use the normalized risk for coloring
    cmap='OrRd',              # Color map (e.g., 'viridis', 'plasma', 'OrRd')
    linewidth=0.5,               # Line width of county borders
    edgecolor='black',           # Color of county borders
    legend=True,                 # Show a legend for the color scale
    legend_kwds={'label': "Normalized Infection Risk", 'orientation': "horizontal", 'fraction': 0.036, 'pad':0.1},
    ax=ax                        # Plot on the specified axis
)
plot_gdf_source.plot(ax=ax, color='blue', markersize= 5, alpha= 1, label = 'Seed')
# Set a title for the map
ax.set_title('Normalized Infection Risk by County', fontsize=15)
ax.legend()
# Remove axes for a cleaner look
ax.set_axis_off()
plt.title(plot_title)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc='lower right') 
plt.savefig(fig_path, dpi = 600)
# Step 4: Display the plot
plt.show()

NameError: name 'plt' is not defined